In [2]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile
import yaml

from loguru import logger

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm
import pandas as pd

import goli

Using backend: pytorch


## Training

In [2]:
# Load a config
with open("../expts/config_micro_ZINC.yaml", "r") as file:
    yaml_config = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
training_dir = "/home/hadim/Drive/Data/goli/test-training"

# Tweak config and paths
yaml_config["datamodule"]["args"]["df_path"] = "../goli/data/micro_ZINC/micro_ZINC.csv"
yaml_config["datamodule"]["args"]["cache_data_path"] = None

yaml_config["trainer"]["trainer"]["min_epochs"] = 1
yaml_config["trainer"]["trainer"]["max_epochs"] = 1

yaml_config["trainer"]["logger"]["save_dir"] = training_dir
yaml_config["trainer"]["model_checkpoint"]["dirpath"] = None
yaml_config["trainer"]["trainer"]["default_root_dir"] = training_dir

In [4]:
# Load datamodule
datamodule = goli.config.load_datamodule(yaml_config)

# Initialize the network
model_class, model_kwargs = goli.config.load_architecture(
    yaml_config,
    in_dim_nodes=datamodule.num_node_feats_with_positional_encoding,
    in_dim_edges=datamodule.num_edge_feats,
)

# Init trainer
metrics = goli.config.load_metrics(yaml_config)
predictor = goli.config.load_predictor(yaml_config, model_class, model_kwargs, metrics)
trainer = goli.config.load_trainer(yaml_config)

/home/hadim/Drive/Documents/valence/Platform/Libs/goli/goli/features/spectral.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvecs[comp, :] = this_eigvecs
/home/hadim/Drive/Documents/valence/Platform/Libs/goli/goli/features/spectral.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvals_tile[comp, :] = this_eigvals
2021-06-08 13:34:09.135 | WARNING  | goli.config._loader:load_trainer:124 - Number of GPUs selected is `1`, but will be ignored since no GPU are available on this device
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [5]:
# Training
trainer.fit(model=predictor, datamodule=datamodule)

2021-06-08 13:34:10.118 | INFO     | goli.data.datamodule:prepare_data:347 - Prepare dataset with 1000 data points.


  0%|          | 0/1000 [00:00<?, ?it/s]

/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: RuntimeWarning: Found unsupported keys in the lr scheduler dict: ['mode']
  warnings.warn(*args, **kwargs)

  | Name     | Type           | Params
--------------------------------------------
0 | model    | FullDGLNetwork | 74.4 K
1 | loss_fun | MSELoss        | 0     
--------------------------------------------
74.4 K    Trainable params
0         Non-trainable params
74.4 K    Total params
0.298     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)
/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
 

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Load pretrained (hard coded path)

In [3]:
# Load a config
with open("../expts/config_micro_ZINC.yaml", "r") as file:
    yaml_config = yaml.load(file, Loader=yaml.FullLoader)

In [4]:
training_dir = "/home/hadim/Drive/Data/goli/test-training"

# Tweak config and paths
yaml_config["datamodule"]["args"]["df_path"] = "../goli/data/micro_ZINC/micro_ZINC.csv"
yaml_config["datamodule"]["args"]["cache_data_path"] = None

yaml_config["trainer"]["trainer"]["min_epochs"] = 1
yaml_config["trainer"]["trainer"]["max_epochs"] = 1

yaml_config["trainer"]["logger"]["save_dir"] = training_dir
yaml_config["trainer"]["model_checkpoint"]["dirpath"] = None
yaml_config["trainer"]["trainer"]["default_root_dir"] = training_dir

In [7]:
# Load datamodule
datamodule = goli.config.load_datamodule(yaml_config)

# Load a trainer
trainer = goli.config.load_trainer(yaml_config)

2021-06-08 13:37:06.943 | WARNING  | goli.config._loader:load_trainer:124 - Number of GPUs selected is `1`, but will be ignored since no GPU are available on this device
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [9]:
# Load a pretrained model
model_path = "s3://goli-public/pretrained-models/ZINC-micro-dummy-test.ckpt"
# model_path = "/home/hadim/Drive/Data/goli/test-training/default/version_0/checkpoints/model.ckpt"
predictor = goli.trainer.predictor.PredictorModule.load_from_checkpoint(model_path)

# Inference
results = trainer.predict(predictor, datamodule=datamodule, return_predictions=True)

/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, predict dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Predicting: 0it [00:00, ?it/s]

/home/hadim/local/conda/envs/goli/lib/python3.9/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)


## Load pretrained (from goli available models)

In [16]:
# Load a config
with open("../expts/config_micro_ZINC.yaml", "r") as file:
    yaml_config = yaml.load(file, Loader=yaml.FullLoader)

In [17]:
training_dir = "/home/hadim/Drive/Data/goli/test-training"

# Tweak config and paths
yaml_config["datamodule"]["args"]["df_path"] = "../goli/data/micro_ZINC/micro_ZINC.csv"
yaml_config["datamodule"]["args"]["cache_data_path"] = None

yaml_config["trainer"]["trainer"]["min_epochs"] = 1
yaml_config["trainer"]["trainer"]["max_epochs"] = 1

yaml_config["trainer"]["logger"]["save_dir"] = training_dir
yaml_config["trainer"]["model_checkpoint"]["dirpath"] = None
yaml_config["trainer"]["trainer"]["default_root_dir"] = training_dir

In [21]:
yaml_config["trainer"]["trainer"]["gpus"] = 0

In [7]:
# Load datamodule
datamodule = goli.config.load_datamodule(yaml_config)

# Load a trainer
trainer = goli.config.load_trainer(yaml_config)

2021-06-08 13:37:06.943 | WARNING  | goli.config._loader:load_trainer:124 - Number of GPUs selected is `1`, but will be ignored since no GPU are available on this device
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [18]:
# Load a pretrained model
predictor = goli.trainer.PredictorModule.load_pretrained_models("ZINC-micro-dummy-test")

# Inference
results = trainer.predict(predictor, datamodule=datamodule, return_predictions=True)

Predicting: 0it [00:00, ?it/s]

In [38]:
type(predictor)

goli.trainer.predictor.PredictorModule

In [31]:
assert tuple(results[0].shape) == (128, 1)